In [1]:
from ast import literal_eval
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.models import ldamodel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.test.utils import common_corpus
from gensim.test.utils import common_texts
from gensim.utils import simple_preprocess
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from statistics import mean
import gensim
import math
import nltk
import numpy as np
import os
import pandas as pd
import random
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
overview = pd.read_csv("debate_data/data_overviews/overview_1.csv")

In [3]:
def string_2_row(df_w_col_name):
    count = 0
    for row in df_w_col_name:
        df_w_col_name[count] = literal_eval(row)
        count+=1
        
string_2_row(overview["moderator"])

In [4]:
# create stemmer and lemmatizer
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

In [5]:
# list of stop words
removed_words = ["ok","yes","right","good","great","hi","look","evening","well","activ","except",\
                "think","differ","style","whatev","go","comment","single","sure","no"]
nltk_stop = list(set(stopwords.words('english')))
political_stop_words = ["president", "american", "america", "govern", "government", "campaign","secretary",\
                        "country","applause","question","governor", "congresswoman", "congress",\
                       "congressman", "partisan", "republican", "democrat", "DNC","GOP","liberal",\
                       "conservative", "senate", "state", "left","right", "nomination","parti","party",\
                       "democraci","constitut","politician","political","politics","pols","politi", "secretari"]
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', \
          'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', \
          'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Montana', 'Nebraska', 'Nevada', \
          'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', \
          'Oklahoma', 'Oregon', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',\
          'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 
          'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

splitted_states = []
for s in states:
    for x in s.split(" "):
        splitted_states.append(x)
        
words = list(set(removed_words + nltk_stop + political_stop_words + splitted_states))
stop_words = list(set(words + [x.upper() for x in words] + [x.capitalize() for x in words]))

In [6]:
# renaming the debates with coder from overview sheet

directory = "debate_data/debates_raw"

for filename in os.listdir(directory):
    
    if filename.endswith(".csv"):
        this_file = pd.read_csv(directory+"/"+filename, encoding = "ISO-8859-1")
        debate_date = (filename.split("-"))[0]        

        coder = overview.loc[overview["date"] == debate_date, "coder"].iloc[0]

        new_name = "debate_data/debates_clean/"+ str(coder) +".csv"
        this_file.to_csv(new_name,index=False)

In [7]:
# multiple steps for cleaning debate data, done below, with output of final cleaned CSV

directory = "debate_data/debates_clean"

for filename in os.listdir(directory):
        
    if filename.endswith(".csv"):
        
        # reading file
        this_file = pd.read_csv(directory+"/"+filename)
        
        debate_code = str(filename.split(".")[0])
                
        # use only last names (eg SEN. OBAMA -> OBAMA)
        who_talks = []
        for name in this_file["speaker"]:
            try:
                who_talks.append(name.split(" ")[-1].upper().replace(".",""))
            except:
                who_talks.append(np.nan)
        this_file["speaker"] = who_talks
        
        two_cols = this_file.reset_index(drop = True)
                
        # make new row for each new sentence   
        row_num = 0
        for row in two_cols["speaker"]:
            two_cols["speaker"][row_num] = str(two_cols["speaker"][row_num]).replace('\n', ' ')
            row_num+=1
            
        row_num = 0
        position = []
        new_data = []
        
        try:
            for x in two_cols["speech"]:
                speaker = two_cols["speaker"][row_num]

                s = sent_tokenize(x)
                if len(s)>1:
                    position.append(row_num)
                    for sent in s:
                        new_data.append([speaker,sent])

                row_num+=1

            new_sent = pd.DataFrame.from_records(new_data)
            new_sent.columns = ["speaker","speech"]
            dropped_para = two_cols.drop(position)

            sent_only = dropped_para.append(new_sent, ignore_index=False)
        except:
            sent_only = two_cols
        
        sent_only = sent_only.reset_index(drop=True)
        
        # removing special characters 
        for x in sent_only["speech"]:
            try:
                t = re.sub('[^a-zA-Z]+', ' ', x) 
                word_tokens = word_tokenize(t) 
                filtered_sentence = [w.lower() for w in word_tokens if not w in stop_words] 
                new_sent = " ".join(filtered_sentence)
                sent_only = sent_only.replace(x,new_sent)
            except:
                pass
        
        # make lower case
        replaced = []
        for row in sent_only["speech"]:
            sentence = []
            row = str(row)
            for word in row.split(" "):
                sentence.append(word.lower())
            as_string = " ".join(sentence)
            replaced.append(as_string)
        sent_only["speech"] = replaced  
        
        # lemmatize words
        replaced = []
        for row in sent_only["speech"]:
            sentence = []
            row = str(row)
            for word in row.split(" "):
                sentence.append(lemmatizer.lemmatize(word))
            as_string = " ".join(sentence)
            replaced.append(as_string)
        sent_only["speech"] = replaced         
        
        # stem words
        replaced = []
        for row in sent_only["speech"]:
            sentence = []
            row = str(row)
            for word in row.split(" "):
                if len(word) > 4:
                    sentence.append(stemmer.stem(word))
            as_string = " ".join(sentence)
            replaced.append(as_string)
        sent_only["speech"] = replaced         
        
        # remove stop words
        replaced = []
        for row in sent_only["speech"]:
            sentence = []
            row = str(row)
            for word in row.split(" "):
                if word not in stop_words:
                    sentence.append(word)
            as_string = " ".join(sentence)
            replaced.append(as_string)
        sent_only["speech"] = replaced
        
        
        # remove speech which is only 2 words or less
        remove = []
        pos = 0
        for row in sent_only["speech"]:
            sentence = []
            row = str(row)
            for word in row.split(" "):
                sentence.append(word)
            if len(sentence) < 3:
                remove.append(pos)
            pos+=1
        
        this_file = sent_only.drop(remove)
        
        # keep only speaker and speech columns
        this_file = this_file[["speaker","speech"]]
        
        # generate new CSV for each debate
        this_file = this_file.reset_index(drop=True)
        new_name = "debate_data/debates_clean/" + str(filename)
        this_file.to_csv(new_name,index=False)

In [8]:
# bit more preprocessing required following visual inspection of data

directory = "debate_data/debates_clean"

for filename in os.listdir(directory):
           
    if filename.endswith(".csv"):

        data = pd.read_csv(directory+"/"+filename)

        # remove rows with no speech in them
        data = data[pd.notnull(data["speech"])]
        data = data.reset_index(drop=True)

        # update CSV for each debate
        new_name = "debate_data/debates_clean/" + str(filename)
        data.to_csv(new_name, index=False)

In [9]:
# some speakers are called "PRESIDENT", rename them

winners = pd.read_csv("debate_data/election_cycle_winners.csv")

directory = "debate_data/debates_clean"

for filename in os.listdir(directory):
    
    if filename.endswith(".csv"):
        
        this_file = pd.read_csv(directory+"/"+filename)
        
        all_names = []
        for row in this_file["speaker"]:
            all_names.append(row)
        all_names = list(set(all_names))
        
        if "PRESIDENT" in all_names:

            debate_code = str(filename.split(".")[0])

            election_year = int(overview.loc[overview["coder"] == debate_code, "election_year"].iloc[0]) - 4

            pres_name = winners.loc[winners["election_year"] == election_year, "G"].iloc[0]

            count1 = 0
            for row in this_file["speaker"]:
                if row == "PRESIDENT":
                    this_file["speaker"][count1] = pres_name.upper()
                count1+=1

            new_file = this_file
            new_name = "debate_data/debates_clean/" + filename
            new_file.to_csv(new_name, index=False)

In [10]:
# function to join subsequent speakers if the same

def join_speakers(data):
    sets = []
    count1 = 1

    for x in data["speaker"]:
        group = [count1-1]
        count2 = count1
        
        while count2 < len(data["speaker"]) and x == data["speaker"][count2]:
            group.append(count2)
            count2+=1
        sets.append(group)
        
        count1+=1
    to_drop = []
    for x in list(range(0,len(sets))):
        try:
            if sets[x][1] == sets[x+1][0]:
                to_drop.append(x+1)
        except:
            pass
    sets = [i for j, i in enumerate(sets) if j not in to_drop]
    for x in sets:
        if len(x)>1:
            for i in x[1:]:
                data["speech"][x[0]] = data["speech"][x[0]] + ". " + data["speech"][i]
    to_keep = list(x[0] for x in sets)
    data = data.ix[to_keep].reset_index(drop=True)
    
    return data

directory = "debate_data/debates_clean"

for filename in os.listdir(directory):
        
    if filename.endswith(".csv"):
        
        # reading file
        data = pd.read_csv(directory+"/"+filename)
        debate_code = str(filename.split(".")[0])
                
        # use only last names (eg SEN. OBAMA -> OBAMA)
        who_talks = []
        for name in data["speaker"]:
            try:
                who_talks.append(name.split(" ")[-1].upper().replace(".",""))
            except:
                who_talks.append(np.nan)
        data["speaker"] = who_talks

        # join sentences when speaker is continously talking
        data = join_speakers(data).reset_index(drop=True)
        
        # check, prints if there is subsequent naming
        check = list(range(0,len(data["speaker"])-1))
        for x in check:
            try:
                if data["speaker"][x] == data["speaker"][x+1]:
                    print(data["speaker"][x], data["speaker"][x+1])
            except:
                print(len(data["speaker"]),x, "*")
                
        # update CSV for each debate
        new_name = "debate_data/debates_clean/" + str(filename)
        data.to_csv(new_name, index=False)

In [11]:
# split files to create moderator and participant CSVs

directory = "debate_data/debates_clean"

for filename in os.listdir(directory):
        
    if filename.endswith(".csv"):
        
        # reading file
        this_file = pd.read_csv(directory+"/"+filename)
        
        debate_code = str(filename.split(".")[0])
        
        # drop people who speak 3 times or less (indicates audience questions)
        extra_names = []
        try:
            filt_data = this_file[this_file.groupby("speaker").speaker.transform("count")<4]
            for row in filt_data["speaker"]:
                extra_names.append(row)            
        except:
            pass
            
        # moderators and audience comments                    
        additional_to_remove = ["(UNKNOWN)","ANNOUNCER", "AUDIENCE","PROTESTER","STUDENT","SAWYER",
                                "Q","Question", "QUESTIONER","APPLAUSE","MODERATOR","Panelists",
                                "OTHERS","OTHER","cartoon","anchor","CANDIDATE","LAUGHTER","Q.",
                                "(LAUGHTER)","LAUGHTER","MALE","female","[","]","(",")","THIS",
                               "happened","George","is","ABC","ALBIN","bachman","bagley","berger",
                               "bream","brooks","Brownstein","Cameron","carolina","cater","Cavuto",
                               "chancellor","clear","correspondent","Cronkite","Crowley","demint",
                               "dinan","DiStaso","do","drummond","economy","epperson","fremd","LEVESQUE",
                               "Goldman","goler","TRUE","You","greenfield","Griffith","hall","Harwood",
                               "hewitt","hiller","Iowa","Jennings","johns","Karl","king","lemon","Liesman",
                                "like","louis","maddow","mcgee","mcmanus","Mears","Morales","morgan",
                                "navarrette","news","niven","novins","obradovich","one","spivak","vancour",
                                "warren","system","shaw","roe","too","truth","votes","register","roberts",
                                "rogers","russert","seib","Spradling","three","vaughn","Wallace","Wickham",
                                "WORKER)","WORKER","yepsen","pelley","reinhard","STEPHANOPOULOS",
                                "unknown","york","raddatz","right","santelli","Strassel","tumulty","wallace",
                               "Albin","Bachman","Cameron","Cavuto","Cramer","Crowley","Cummings",
                               "Demint","Dinan","Distaso","Evans","Fahey","Goldman","Goler","Griffith",
                               "Ham","Harwood","Hewitt","Hiller","Jennings","King","Lemon","Liesman",
                                "Lopez", "Louis","Maddow","Mcelveen","McManus","Morales","Novins",
                                "O'brien","Pelley","Raddatz","Reinhard","Russert","Seib","Shaw","LOPEZ"
                                "Spradling","Stanton","Stephanopoulos","Strassel","Tumulty","UNKNOWN",
                                "Vaughn","Wallace","Williams","Woodruff","Yepsen","York", "MOYERS","Fahey",
                                "KERR","GOODSON","CRAMER","VANOCUR","TASH","FAHEY","JACKSON","TRULUCK",
                                "LOPEZ","LEVESQUE","university","(?)","?",")",'NIXON"',"ARRARÃS","LEHRER"]
            
            
        row_num = overview["coder"][overview["coder"] == debate_code].index
        
        for person in overview["moderator"][row_num]:
            for mod in person:
                additional_to_remove.append(mod)
                
        exclude = ["Bush","Johnson","JACKSON","HUNTER","ROMNEY","Anderson","FIORINA","PERRY","johnson",\
                         "REAGAN","clark","sharpton","dukakis","Ford"]        
        exclude = list(set(exclude + [x.lower() for x in exclude] + [x.upper() for x in exclude] 
                           + [x.capitalize() for x in exclude]))
        
        mods = additional_to_remove + extra_names

        mods = list(set(mods + [x.lower() for x in mods] + [x.upper() for x in mods] 
                        + [x.capitalize() for x in mods]))
        
        
        non_parts = [x for x in mods if x not in exclude]

        pos = []
        
        count = 0
        for x in this_file["speaker"]:
            if x in non_parts:
                pos.append(count)
            count+=1
        
        # pos is the index position of moderators
                
        participant_comments = this_file.drop(pos)
        participant_comments = participant_comments.dropna(axis=0, subset=['speaker'])
        participant_comments = participant_comments.reset_index(drop = True)
        participant_comments = join_speakers(participant_comments)
        part_name = "debate_data/debates_part/" + str(filename)

        moderator_comments = this_file.iloc[pos,:]
        moderator_comments = moderator_comments.dropna(axis=0, subset=['speaker'])
        moderator_comments = moderator_comments.reset_index(drop = True)
        moderator_comments = join_speakers(moderator_comments)
        mod_name = "debate_data/debates_mod/" + str(filename)
        
        participant_comments.to_csv(part_name, index=False)
        moderator_comments.to_csv(mod_name, index=False)

In [12]:
# rename participants in each debate as NAME+YYYY where YYYY stands for election year

directory = "debate_data/debates_part"

for filename in os.listdir(directory):
        
    if filename.endswith(".csv"):
        
        # reading file
        this_file = pd.read_csv(directory+"/"+filename)
        
        debate_code = str(filename.split(".")[0])

        year_code = int(debate_code[1:5])
        
        if year_code % 4 != 0:
            year_code = year_code+1
        
        new_speaker = []
        for row in this_file["speaker"]:
            name = str(row) + str(year_code)
            new_speaker.append(name)
            
        this_file["speaker"] = new_speaker
        
        part_name = "debate_data/debates_part/" + str(filename)
        this_file.to_csv(part_name, index=False)

In [13]:
# add list of participants to the overview file

overview["participants"] = np.nan

directory = "debate_data/debates_part"

participants_ordered = []
for filename in os.listdir(directory):

    if filename.endswith(".csv"):
                
        data = pd.read_csv(directory+"/"+filename)
        
        debate_code = str(filename.split(".")[0])
        
        participants = []
        for row in data["speaker"]:
            participants.append(row)
        participants = list(set(participants))
        
        location = overview["coder"][overview["coder"] == debate_code].index.tolist()
        
        for i in location:
            location = int(i)
        
        overview["participants"][location] = participants
        
overview.to_csv("debate_data/data_overviews/overview_1.csv",index=False)

In [14]:
# create candidate data CSV 

candidate_data = pd.DataFrame(columns = ["participants","year","part_code"])

all_p = []
for row in overview["participants"]:
    for x in row:
        all_p.append(x)
all_p = list(set(all_p))

name = []
year = []
part_code = []
for x in all_p:
    name.append(x[:-4])
    year.append(x[-4:])
    part_code.append(x)
candidate_data["participants"] = name
candidate_data["year"] = year
candidate_data["part_code"] = part_code

candidate_data.to_csv("debate_data/data_overviews/candidate_data_1.csv",index=False)

In [17]:
# add background data to candidates

candidate_background = pd.read_csv("debate_data/data_overviews/candidate_background.csv")

candidate_data["election_age"] = np.nan
candidate_data["gender"] = np.nan
candidate_data["party"] = np.nan
candidate_data["general"] = np.nan
candidate_data["affiliation"] = np.nan

non_candidates_to_drop = []

for x in candidate_data["part_code"]:
    
    location1 = candidate_background["part_code"][candidate_background["part_code"] == x].index.tolist()
    for i in location1:
            location1 = int(i)
    
    location2 = candidate_data["part_code"][candidate_data["part_code"] == x].index.tolist()
    for i in location2:
            location2 = int(i)
    try:        
        candidate_data["election_age"][location2] = candidate_background["election_age"][location1]
        candidate_data["gender"][location2] = candidate_background["gender"][location1]
        candidate_data["party"][location2] = candidate_background["party"][location1]
        candidate_data["general"][location2] = candidate_background["general"][location1]
        candidate_data["affiliation"][location2] = candidate_background["affiliation"][location1]
    except:
        non_candidates_to_drop.append(x)

In [41]:
candidate_data = candidate_data[~candidate_data['part_code'].isin(non_candidates_to_drop)].reset_index(drop=True)
        
candidate_data["n_debates"] = np.nan

all_parts = []

for x in overview["participants"]:
    for i in x:
        all_parts.append(i)

for x in all_parts:
    location1 = candidate_data["part_code"][candidate_data["part_code"] == x].index.tolist()
    for i in location1:
            location1 = int(i)
    candidate_data["n_debates"][i] = all_parts.count(x)
    
count = 0
for x in overview["participants"]:
    parts = []
    for i in x:
        parts.append(i)
    updated_parts = [x for x in parts if x not in non_candidates_to_drop]
    overview["participants"][count] = updated_parts
    count+=1

In [42]:
# create file for each speaker
speakers = pd.read_csv("debate_data/data_overviews/candidate_data_1.csv")       
        
for row in speakers["part_code"]:
    new_file = pd.DataFrame(columns = ["speaker","speech"])
    new_name = "debate_data/speakers/" + str(row) +".csv"
    new_file.to_csv(new_name,index=False)
    
# join all the debates to one csv
all_part_debates = pd.DataFrame(columns = ["speaker","speech"])

directory = "debate_data/debates_part"
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        data = pd.read_csv(directory+"/"+filename)
        all_part_debates = pd.concat([all_part_debates, data], ignore_index=True)         

# fill speaker files
for coder in candidate_data["part_code"]:

    new_file = all_part_debates.loc[all_part_debates.speaker == coder]
    new_file = new_file.reset_index(drop = True)
    new_name = "debate_data/speakers/" + str(coder) +".csv"
    new_file.to_csv(new_name,index=False)

In [62]:
candidate_data["total_words"] = np.nan
candidate_data["unique_words"] = np.nan

directory = "debate_data/speakers"

for filename in os.listdir(directory):

    if filename.endswith(".csv"):
                
        data = pd.read_csv(directory+"/"+filename)
        
        debate_code = str(filename.split(".")[0])
        
        words = []

        for x in data["speech"]:
            for i in x.split(" "):
                words.append(i)
                
        loc = candidate_data["part_code"][candidate_data["part_code"] == debate_code].index
        n_words = len(words)
        candidate_data["total_words"][loc] = n_words
        candidate_data["unique_words"][loc] = len(set(words))

In [69]:
# update candidate_data_1
candidate_data.to_csv("debate_data/data_overviews/candidate_data_1.csv",index=False)
candidate_data

,participants,year,part_code,election_age,gender,party,general,affiliation,n_debates,total_words,unique_words
0,QUAYLE,1988,QUAYLE1988,41.0,0.0,1.0,1.0,rep,1.0,1391.0,644.0
1,BENTSEN,1988,BENTSEN1988,67.0,0.0,1.0,0.0,dem,1.0,1417.0,675.0
2,O'MALLEY,2016,O'MALLEY2016,53.0,0.0,0.0,0.0,dem,5.0,4703.0,1478.0
3,JOHNSON,2012,JOHNSON2012,59.0,0.0,0.0,0.0,rep,1.0,158.0,113.0
4,GORE,1996,GORE1996,48.0,0.0,1.0,1.0,dem,1.0,1633.0,755.0
...,...,...,...,...,...,...,...,...,...,...,...
96,DUKAKIS,1988,DUKAKIS1988,55.0,0.0,1.0,0.0,dem,2.0,3434.0,1147.0
97,BRADLEY,2000,BRADLEY2000,57.0,0.0,0.0,0.0,dem,8.0,8145.0,2024.0
98,MCCAIN,2008,MCCAIN2008,82.0,0.0,1.0,0.0,rep,18.0,12987.0,2685.0
99,LIEBERMAN,2004,LIEBERMAN2004,62.0,0.0,0.0,0.0,dem,2.0,746.0,443.0
